## Reading and Writing .CSV or .TSV Files

In [ ]:
#Reading
path_au = "s3://ibt-global-phrasedoc/qag/v1/filtered/marketplace_id=111172/date=20220301/"
df_au = spark.read.option('delimiter', '\t').csv(path_au).selectExpr("_c0 as asin", "_c1 as marketplace_id" , "_c2 as query", "_c3 as phrase_doc_score")

#Writing
path_ibt = "s3://span-dev-users/dgoyl/Try_EMR/using_emb/conflated_output_17M_qba_hybrid_distribution_"\
                    + t_string + "/block_size_" + str(max_block_size) + "_IBT_Entries_Number_Of_Chunks_25"
df_au_filtered_sentinel.repartition(25).write.csv(path_ibt, mode = "overwrite", sep = '\t', header=True) # Wrtie Tab Separated File                                                                        


## Reading and Writing .Parquet Files

In [ ]:
#Reading
path_pseudo_pairs = "s3://span-dev-users/dgoyl/Try_EMR/using_emb/conflated_output_17M_qba_hybrid_distribution_" + t_string + "/block_size_" + str(max_block_size) + "_matchedRecords_pseudo"
df_pairs = spark.read.parquet(path_pseudo_pairs)



#Writing
path_ibt = "s3://span-dev-users/dgoyl/Try_EMR/using_emb/conflated_output_17M_qba_hybrid_distribution_"\
                    + t_string + "/block_size_" + str(max_block_size) + "_IBT_Entries_Number_Of_Chunks_25"
df_au_filtered_sentinel.repartition(25).write.parquet(path_ibt_parquet, mode="overwrite")

## GroupBy and Collect

In [ ]:
from pyspark.sql import functions as F
df_au_list = df_au_filtered.groupby("query").agg(F.collect_set("asin"))\
                           .withColumnRenamed('collect_set(asin)', "asin_list").cache()

## GroupBy and Count and Sort

In [ ]:
from pyspark.sql.functions import col
df_group = df.groupBy('keyword').count().sort(col("count").desc())

## Merge Columns

In [ ]:
from pyspark.sql.functions import col
import pyspark.sql.functions as f
total_list = [f.col("keywords")] + [f.col("parent_index")] + [f.col("parent_score")] + [f.col("parent_level")]
df_qba_list = df_qba_refined_parent.withColumn("List_Total", f.array(total_list))

## Split Column Into Multiple Columns

In [ ]:
df_split_nodes = df_all_nodes.select(df_all_nodes.Nodes[0].alias('keywords'), \
                                         df_all_nodes.Nodes[1].alias('Threshold_Browse_Index'),\
                                         df_all_nodes.Nodes[2].alias('Threshold_Browse_Score'),\
                                         df_all_nodes.Nodes[3].alias('Threshold_Browse_Level'),\
                                        )

## Exploding Lists Into Rows

In [ ]:
from pyspark.sql.functions import explode
df_ibt_explode = df_ibt_list.withColumn('keywords',explode('list_of_keywords'))

## Using UDFs

In [ ]:
from pyspark.sql.functions import col,udf
from pyspark.sql import types as T

def extract_nodes(input_array):
    temp_listi = input_array[1:41]    # i denotes index
    temp_lists = input_array[41:81]   # s denotes score
    temp_listl = input_array[81:121]  # l denotes level
    parent_index = input_array[121] #browse node in which query is currently in
    parent_score = input_array[122] #browse node in which query is currently in
    parent_level = input_array[123] #browse node in which query is currently in

    level_node_info = []
    index = 0
    count = 0
    for level in temp_listl:
        if level == str(browse_node_level) and float(temp_lists[index]) >= threshold[browse_node_level-1]:
            node_info = [input_array[0], temp_listi[index], temp_lists[index], level]
            count +=1
            level_node_info.append(node_info)
        index +=1

    if count == 0:
        node_info = [input_array[0], parent_index, parent_score, parent_level]
        level_node_info.append(node_info)

    return level_node_info

leaf_udf = udf(lambda z:extract_nodes(z),T.ArrayType(T.ArrayType(T.StringType())))   
df_qba_list_2 = df_qba_list.withColumn('Keyword_Level_Nodes',leaf_udf(col("List_Total")))